In [2]:
import click

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.losses import CategoricalCrossentropy
import matplotlib.pyplot as plt
import numpy as np
import dataloader

from loguru import logger

import importlib

import tf_models  # First, import the library
importlib.reload(tf_models)  # Now, reload it
from tf_models import *


dataset = 'cifar10'


loader = getattr(dataloader, f"load_{dataset}")
x_train, y_train, x_test, y_test = loader(onehot=True)

if x_train[0].ndim == 2:
    x_train = x_train[..., np.newaxis]
    x_test = x_test[..., np.newaxis]
image_shape = x_train[0].shape  # (28, 28)

model = ResNet(image_shape, num_classes=y_train.shape[1], l2_lambda=4e-4)

model.compile(
    optimizer="adam", loss=CategoricalCrossentropy(), metrics=["accuracy"]
)
if False:
    model.load_weights(f'{dataset}-viz.keras')
logger.info('done')

2023-10-10 02:36:18.934 | INFO     | dataloader:_load_keras:30 - Using cifar10 dataset
2023-10-10 02:36:18.935 | INFO     | dataloader:_load_keras:33 - The size is 50000
2023-10-10 02:36:18.936 | INFO     | dataloader:_load_keras:34 - The shape is: (32, 32, 3)
2023-10-10 02:36:20.547 | INFO     | __main__:<module>:40 - done


In [3]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv2d_6 (Conv2D)           (None, 32, 32, 64)           1792      ['input_2[0][0]']             
                                                                                                  
 batch_normalization_6 (Bat  (None, 32, 32, 64)           256       ['conv2d_6[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 re_lu_4 (ReLU)              (None, 32, 32, 64)           0         ['batch_normalization_6[0]

In [4]:
stage = 'test'

if stage == 'test':
    iter = 4
    epo = 8
else:
    iter = 32
    epo = 16
for i in range(iter):
    with tf.device("/GPU:0"):
        history = model.fit(
            x_train, y_train, epochs=epo, batch_size=64, validation_split=0.02
        )
        model.save(f'{dataset}-viz.keras')

    # Evaluate the model on the test set
    logger.info(f"iter {i}")
    loss, accuracy = model.evaluate(x_test, y_test)
    logger.info(f"Test loss: {loss:.4f}")
    logger.info(f"Test accuracy: {accuracy:.4f}")

Epoch 1/8


2023-10-10 02:36:43.181415: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-10-10 02:36:45.664719: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8900
2023-10-10 02:36:46.712192: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55ea4db87200 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-10 02:36:46.712236: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-SXM2-16GB, Compute Capability 7.0
2023-10-10 02:36:46.719816: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-10 02:36:46.877562: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled clu

766/766 [==============================] - 60s 36ms/step - loss: 3.3941 - accuracy: 0.4015 - val_loss: 2.7430 - val_accuracy: 0.4810
Epoch 2/8
766/766 [==============================] - 27s 35ms/step - loss: 2.2373 - accuracy: 0.5677 - val_loss: 1.8906 - val_accuracy: 0.6220
Epoch 3/8
766/766 [==============================] - 27s 35ms/step - loss: 1.7037 - accuracy: 0.6376 - val_loss: 1.8270 - val_accuracy: 0.5730
Epoch 4/8
766/766 [==============================] - 27s 35ms/step - loss: 1.4307 - accuracy: 0.6877 - val_loss: 1.6765 - val_accuracy: 0.6120
Epoch 5/8
766/766 [==============================] - 27s 35ms/step - loss: 1.2767 - accuracy: 0.7194 - val_loss: 1.4584 - val_accuracy: 0.6470
Epoch 6/8
766/766 [==============================] - 27s 35ms/step - loss: 1.1610 - accuracy: 0.7456 - val_loss: 1.3559 - val_accuracy: 0.6960
Epoch 7/8
766/766 [==============================] - 27s 35ms/step - loss: 1.0864 - accuracy: 0.7638 - val_loss: 1.5051 - val_accuracy: 0.6290
Epoch 8/8

2023-10-10 02:40:42.901 | INFO     | __main__:<module>:17 - iter 0


313/313 [==============================] - 2s 7ms/step - loss: 1.5547 - accuracy: 0.6192


...

2023-10-10 02:47:58.873 | INFO     | __main__:<module>:19 - Test loss: 1.0505
2023-10-10 02:47:58.874 | INFO     | __main__:<module>:20 - Test accuracy: 0.7613


Epoch 1/8
766/766 [==============================] - 27s 35ms/step - loss: 0.6919 - accuracy: 0.8711 - val_loss: 0.8153 - val_accuracy: 0.8400
Epoch 2/8
766/766 [==============================] - 26s 34ms/step - loss: 0.6821 - accuracy: 0.8725 - val_loss: 1.1831 - val_accuracy: 0.7360
Epoch 3/8
766/766 [==============================] - 27s 35ms/step - loss: 0.6785 - accuracy: 0.8737 - val_loss: 1.1185 - val_accuracy: 0.7550
Epoch 4/8
766/766 [==============================] - 26s 34ms/step - loss: 0.6680 - accuracy: 0.8775 - val_loss: 0.9009 - val_accuracy: 0.7900
Epoch 5/8
766/766 [==============================] - 27s 35ms/step - loss: 0.6714 - accuracy: 0.8778 - val_loss: 0.8747 - val_accuracy: 0.8260
Epoch 6/8
766/766 [==============================] - 26s 34ms/step - loss: 0.6603 - accuracy: 0.8804 - val_loss: 0.9065 - val_accuracy: 0.8060
Epoch 7/8
766/766 [==============================] - 26s 34ms/step - loss: 0.6615 - accuracy: 0.8803 - val_loss: 0.8071 - val_accuracy: 0.8450

2023-10-10 02:51:32.568 | INFO     | __main__:<module>:17 - iter 3


313/313 [==============================] - 2s 7ms/step - loss: 0.8314 - accuracy: 0.8211


2023-10-10 02:51:35.081 | INFO     | __main__:<module>:19 - Test loss: 0.8314
2023-10-10 02:51:35.082 | INFO     | __main__:<module>:20 - Test accuracy: 0.8211


In [ ]:
model.save(f'{dataset}-viz.keras')

In [ ]:
model.load_weights(f'{dataset}-viz.keras')